## Setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from __future__ import division

import numpy as np
import os
from glob import glob

from PIL import Image
from copy import deepcopy

from IPython.display import clear_output

from sklearn import linear_model, datasets, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set_context('poster')
colors = sns.color_palette("cubehelix", 5)
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import pandas as pd

import scipy.stats as stats
from scipy.stats import norm
import sklearn

from importlib import reload

In [ ]:
## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'..','..')) ## use relative paths
data_dir = os.path.abspath(os.path.join(curr_dir,'..','..','data')) ## use relative paths 'D:\\data'
plot_dir =  os.path.abspath(os.path.join(curr_dir,'..','..','results','plots'))
csv_dir = os.path.join(proj_dir, 'results','csv')
nb_name = '1_object_evidence_during_recognition'

## add helpers to python path
import sys
if os.path.join(proj_dir, 'analysis','utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'analysis','utils'))

## module definitions
import object_evidence_analysis_helpers as utils
reload(utils)
utils.data_dir = data_dir
utils.path_to_recog = os.path.join(data_dir, 'features/recognition')
utils.path_to_draw = os.path.join(data_dir, 'features/production')
utils.roi_list_draw = np.array(['V1Draw', 'V2Draw', 'LOCDraw', 'InsulaDraw', 'preCentralDraw','ParietalDraw'])
utils.roi_list_recog = ['V1','V2','LOC','IT','fusiform','parahippo','PRC','ento','hipp']
utils.roi_list_recog_formatted = np.array(['V1', 'V2', 'LOC', 'FUS','PHC','IT','ENT','PRC','HC']) 
roi_list_recog = utils.roi_list_recog

### get file list

In [ ]:
## get raw file list for recognition runs
path_to_recog = utils.path_to_recog

RECOG_METAS = sorted([i for i in os.listdir(path_to_recog) if (i.split('.')[-1]=='csv') & (i.split('_')[2][-4:] != 'Draw')])
RECOG_FEATS = sorted([i for i in os.listdir(path_to_recog) if (i.split('.')[-1]=='npy') & (i.split('_')[1][-4:] != 'Draw')])
RECOG_SUBS = np.array([i.split('_')[0] for i in RECOG_FEATS])

recog_sub_list = np.unique(RECOG_SUBS)

def preprocess_recog(RECOG_METAS, RECOG_FEATS):
    M = [i for i in RECOG_METAS if len(i.split('.')[0].split('_'))==4]
    F = [i for i in RECOG_FEATS if len(i.split('.')[0].split('_'))==4]
    return M,F

RECOG_METAS, RECOG_FEATS = preprocess_recog(RECOG_METAS, RECOG_FEATS)

In [ ]:
## get raw file list for drawing runs
path_to_draw = utils.path_to_draw

DRAW_METAS = sorted([i for i in os.listdir(path_to_draw) if (i.split('.')[-1]=='csv')])
DRAW_FEATS = sorted([i for i in os.listdir(path_to_draw) if (i.split('.')[-1]=='npy')])
DRAW_SUBS = np.array([i.split('_')[0] for i in DRAW_FEATS])
draw_sub_list = np.unique(DRAW_SUBS)

In [ ]:
## get subject ID's that have complete datasets from all phases of experiment
sub_list = np.intersect1d(recog_sub_list,draw_sub_list)
print('Number of subs: {}'.format(len(sub_list)))

In [ ]:
## filter file list so only contains the sessions that have full datasets
def extract_good_sessions(DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS):
    _DRAW_METAS = [i for i in DRAW_METAS if i.split('_')[1] in sub_list]
    _DRAW_FEATS = [i for i in DRAW_FEATS if i.split('_')[0] in sub_list]
    _RECOG_METAS = [i for i in RECOG_METAS if i.split('_')[1] in sub_list]
    _RECOG_FEATS = [i for i in RECOG_FEATS if i.split('_')[0] in sub_list]
    return _DRAW_METAS, _DRAW_FEATS, _RECOG_METAS, _RECOG_FEATS

DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS =  \
extract_good_sessions(DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS)

RECOG_SUBS = np.array([i.split('_')[0]+'_neurosketch' for i in RECOG_FEATS])
RECOG_ROIS = np.array([i.split('_')[1] for i in RECOG_FEATS])

DRAW_SUBS = np.array([i.split('_')[0]+'_neurosketch' for i in DRAW_FEATS])
DRAW_ROIS = np.array([i.split('_')[1] for i in DRAW_FEATS])

In [ ]:
print('We have data from {} subjects.'.format(len(sub_list)))

### PRODUCTION: How well do we do at classifying the target during production runs when we train on initial recognition patterns only?

In [ ]:
reload(utils)
version = '4way'
logged = True

really_run = 0
if really_run:

    ALLDM, Acc = utils.make_drawing_predictions(sub_list,roi_list_recog,
                                            version=version,logged=logged)
    ## save out ALLDM & Acc
    Acc = np.array(Acc)
    np.save(os.path.join(csv_dir,'object_classifier_accuracy_production.npy'),Acc)
    ALLDM.to_csv(os.path.join(csv_dir,'object_classifier_logprobs_production.csv'),index=False)
    
else:
    ## load in existing ALLDM & Acc 
    Acc = np.load(os.path.join(csv_dir,'object_classifier_accuracy_production.npy'))
    ALLDM = pd.read_csv(os.path.join(csv_dir,'object_classifier_logprobs_production.csv'))
    
print('Done!')

### RECOGNITION: How well do we do at classifying the target during the pre/post recognition runs, trained on initial recognition data?

In [ ]:
reload(utils)
logged = True
version = '4way'

really_run = 0
if really_run:

    ALLPRE, Accpre = utils.make_prepostrecog_predictions_withinphase(sub_list,roi_list_recog,test_phase='pre',logged=logged)
    ALLPOST, Accpost = utils.make_prepostrecog_predictions_withinphase(sub_list,roi_list_recog,test_phase='post',logged=logged)

    ## add raw probs to ALLPRE and ALLPOST dataframes
    append_raw_dict = dict(zip(['t1_prob','t2_prob','c1_prob','c2_prob'],['t1_prob_raw','t2_prob_raw','c1_prob_raw','c2_prob_raw']))
    for key, value in append_raw_dict.iteritems():
        ALLPRE[value] = ALLPRE[key].apply(lambda x: np.exp(x))
        ALLPOST[value] = ALLPOST[key].apply(lambda x: np.exp(x))

    ## save out dataframe
    ALLPRE.to_csv(os.path.join(csv_dir,'object_classifier_probs_pre.csv'),index=False) ## train recog, test drawing run    
    ALLPOST.to_csv(os.path.join(csv_dir,'object_classifier_probs_post.csv'),index=False) ## train recog, test drawing run    
    
    ## save out Acc
    np.save(os.path.join(csv_dir,'object_classifier_accuracy_pre.npy'),Accpre)
    np.save(os.path.join(csv_dir,'object_classifier_accuracy_post.npy'),Accpost)
    Accdiff = np.array(Accpost) - np.array(Accpre)
    np.save(os.path.join(csv_dir,'object_classifier_accuracy_diff.npy'),Accdiff)
    
else:
    ## load in existing ALLPRE, ALLPOST and Acc dataframes
    ALLPRE = pd.read_csv(os.path.join(csv_dir,'object_classifier_probs_pre.csv'))
    ALLPOST = pd.read_csv(os.path.join(csv_dir,'object_classifier_probs_post.csv'))
    
    Accpre = np.load(os.path.join(csv_dir,'object_classifier_accuracy_pre.npy'))
    Accpost = np.load(os.path.join(csv_dir,'object_classifier_accuracy_post.npy'))
    Accdiff = np.load(os.path.join(csv_dir,'object_classifier_accuracy_diff.npy'))    

print('Done!')


### plot object decoding accuracy across pre, production, and post phases

In [ ]:
reload(utils)

## aggregate pre, draw, post decoding accuracy
AE = utils.preprocess_acc_array(Accpre,phase='pre')
AD = utils.preprocess_acc_array(Acc,phase='draw')
AO = utils.preprocess_acc_array(Accpost,phase='post')
AF = utils.preprocess_acc_array(Accdiff,phase='diff')

## save out change in decoding accuracy from pre to post to dataframe
A = pd.concat([AE,AD,AO],axis=0)
A2 = pd.concat([AE,AD,AO,AF],axis=0)
A2.to_csv(os.path.join(csv_dir,'object_classifier_accuracy_allphases.csv'))

## generate plot
g = sns.catplot(data=A,
            x='roi',
            y='acc',
            hue='phase',
            kind='bar',
            hue_order=['pre','post','draw'],
            palette='deep',
            legend=False,
            aspect=2,
            height=6)
plt.legend(bbox_to_anchor=(1.03,1))
plt.ylabel('object decoding accuracy')
plt.xlabel('')
plt.axhline(0.25,linestyle=':',color='k')
out_path = os.path.join(plot_dir,'{}/object_classifier_accuracy_bars_allphases.pdf'.format(nb_name))
plt.savefig(out_path,format='pdf')

In [ ]:
## print 95% CIs for PRE phase decoding accuracy for each ROI
reload(utils)
roi_list_formatted = ['V1', 'V2', 'LOC', 'FUS', 'PHC', 'IT', 'ENT', 'PRC', 'HC']
num_iters = 10000
print('PRE')
print('Number of boostrap iterations = {}'.format(num_iters))
for i,roi in enumerate(roi_list_formatted):
    U,lb,ub,p1,p2 = utils.bootstrapCI(Accpre[i,:],nIter=num_iters,crit_val=0.25)
    mu = np.round(np.mean(Accpre[i,:]),3)
    print('ROI:  {}| mu = {} | 95% CI: [{}, {}] | p = {}'.format(roi, mu, np.round(lb,3), np.round(ub,3), p1))
    
    

In [ ]:
## print 95% CIs for POST phase decoding accuracy for each ROI
reload(utils)
roi_list_formatted = ['V1', 'V2', 'LOC', 'FUS', 'PHC', 'IT', 'ENT', 'PRC', 'HC']
num_iters = 10000
print('POST')
print('Number of boostrap iterations = {}'.format(num_iters))
for i,roi in enumerate(roi_list_formatted):
    U,lb,ub,p1,p2 = utils.bootstrapCI(Accpost[i,:],nIter=10000,crit_val=0.25)
    mu = np.round(np.mean(Accpost[i,:]),3)
    print('ROI:  {}| mu = {} | 95% CI: [{}, {}] | p = {}'.format(roi, mu, np.round(lb,4), np.round(ub,4), p1))


In [ ]:
## print 95% CIs for change in decoding accuracy for each ROI
reload(utils)
roi_list_formatted = ['V1', 'V2', 'LOC', 'FUS', 'PHC', 'IT', 'ENT', 'PRC', 'HC']
num_iters = 10000
print('POST-PRE')
print('Number of boostrap iterations = {}'.format(num_iters))
for i,roi in enumerate(roi_list_formatted):
    U,lb,ub,p1,p2 = utils.bootstrapCI(Accdiff[i,:],nIter=10000,crit_val=0)
    mu = np.round(np.mean(Accdiff[i,:]),4)    
    print('ROI:  {}| mu = {} | 95% CI: [{}, {}] | p = {}'.format(roi, mu, np.round(lb,4), np.round(ub,4), p1))


In [ ]:
### pretty formatting for easier pasting into paper
## print 95% CIs for change in decoding accuracy for each ROI
reload(utils)
roi_list_formatted = ['V1', 'V2', 'LOC', 'FUS', 'PHC', 'IT', 'ENT', 'PRC', 'HC']
num_iters = 10000
print('POST')
print('Number of boostrap iterations = {}'.format(num_iters))
for i,roi in enumerate(roi_list_formatted):
    U,lb,ub,p1,p2 = utils.bootstrapCI(Accpost[i,:],nIter=10000,crit_val=0)
    print('{}: [{}, {}],'.format(roi, str(np.round(lb,3))[1:], str(np.round(ub,3))[1:]))


### add handy columns to dataframe (e.g., target probability)

In [ ]:
## note, this cell takes a while to run ... 
reload(utils)
ALLPRE = utils.add_target_prob_column(ALLPRE)
ALLPOST = utils.add_target_prob_column(ALLPOST)
print('Done!')

In [ ]:
## post processing on the mega dataframes for pretty plotting
ALLPRE['phase'] = 'pre'
ALLPOST['phase'] = 'post'
ALLPREPOST = pd.concat([ALLPRE,ALLPOST],axis=0)

## add formatted ROI list
roi_list = ['V1','V2','LOC','IT','fusiform','parahippo','PRC','ento','hipp']
roi_list_formatted = ['V1', 'V2', 'LOC', 'FUS', 'PHC', 'IT', 'ENT', 'PRC', 'HC']
roi_formatter = dict(zip(roi_list,roi_list_formatted))
ALLPREPOST['roi_formatted'] = ALLPREPOST['roi'].apply(lambda x: roi_formatter[x])

## add max prob column to dataframe
ALLPREPOST['max_prob_raw'] = ALLPREPOST[['t1_prob_raw','t2_prob_raw','c1_prob_raw','c2_prob_raw']].max(axis=1)

## add correct column, assuming classifier deterministically takes top probability object
ALLPREPOST['correct'] = ALLPREPOST['max_prob_raw']==ALLPREPOST['target_prob_raw']

In [ ]:
## define accuracy dataframe for each roi, subject, phase, roi
ALLPREPOSTACC = ALLPREPOST.groupby(['roi_formatted','subj','phase','trained'])['correct'].mean().reset_index()
ALLPREPOSTACC.to_csv(os.path.join(csv_dir,'object_classifier_prepost_accuracy.csv'),index=False)

## define mean target probability dataframe for each roi, subject, phase, roi
ALLPROBS = ALLPREPOST.groupby(['roi_formatted','subj','phase','trained'])['target_prob_raw'].mean().reset_index()
ALLPROBS['foil_prob_raw'] = ALLPREPOST.groupby(['roi_formatted','subj','phase','trained'])['foil_prob_raw'].mean().reset_index()['foil_prob_raw']
ALLPROBS.to_csv(os.path.join(csv_dir,'object_classifier_prepost_rawprobs.csv'),index=False)

In [ ]:
g = sns.catplot(data=ALLPREPOST, 
            x = 'roi_formatted',
            y = 'target_prob_raw',
            hue = 'phase',
            col = 'trained',
            col_order = [True,False],
            kind = 'bar',
            legend=False,
            aspect = 1.5,
            palette = 'husl')

def refline(*args, **kwargs):
    plt.axhline(0.25,linestyle=':',color='k')    

g = g.map(refline)    
g = (g.set_axis_labels("", 'classifier probability'))
plt.legend()
    

In [ ]:
g = sns.catplot(data=ALLPREPOSTACC, 
            x = 'roi_formatted',
            y = 'correct',
            hue = 'phase',
            col = 'trained',
            col_order = [True,False],
            order = roi_list_formatted,
            hue_order = ['pre','post'],
            kind = 'bar',
            legend=False,
            aspect = 1.5,
            palette = 'husl')

def refline(*args, **kwargs):
    plt.axhline(0.25,linestyle=':',color='k')    

g = g.map(refline)    
g = (g.set_axis_labels("", 'object decoding accuracy'))
plt.legend()

### generate object classifier prediction dataframes and save to file

In [ ]:
reload(utils)
dpre = utils.get_log_odds(ALLPRE,
               this_iv = 'time_point',                          
               roi_list = roi_list_recog,
               phase = 'pre',
               version='4way',
               logged=True,
               proj_dir=proj_dir)

In [ ]:
reload(utils)
dpre = utils.get_log_odds(ALLPOST,
               this_iv = 'time_point',
               roi_list = roi_list_recog,
               phase = 'post',
               version='4way',
               logged=True,
               proj_dir=proj_dir)